## Question 1

The given conditions of the question are :
$$f_X(x)=\frac{p}{\sqrt{2\pi \sigma_1^2}}\exp\left(-\frac{(x-\mu_1)^2}{2\sigma_1^2}\right)+\frac{1-p}{\sqrt{2\pi \sigma_2^2}}\exp\left(-\frac{(x-\mu_2)^2}{2\sigma_2^2}\right)$$

$$p = 0.88, \mu_1 = 3.79,\mu_2 = 32.64, \sigma_1 = 8.08, \sigma_2 = 7.39 $$

$$ stepsize =0.002, \epsilon = 1/10^5, iterations = 10000$$


From then given pdf and the corresponding log-likelihood, we get the following gradient:

$$\begin{align}
   \triangledown f(x)
      =   
    \begin{bmatrix}
           \sum_{i=1}^n \frac{1}{f_X(x_i)}[\frac{1}{\sqrt{2\pi \sigma_1^2}}\exp\left(-\frac{(x_i-\mu_1)^2}{2\sigma_1^2}\right) - \frac{1}{\sqrt{2\pi \sigma_2^2}}\exp\left(-\frac{(x_i-\mu_2)^2}{2\sigma_2^2}\right)]\\
           \sum_{i=1}^n \frac{1}{f_X(x_i)}\frac{p}{\sqrt{2\pi \sigma_1^2}}\exp\left(-\frac{(x_i-\mu_1)^2}{2\sigma_1^2}\right)\left(\frac{x_i-\mu_1}{\sigma_1^2}\right)\\
           \sum_{i=1}^n \frac{1}{f_X(x_i)}\frac{1-p}{\sqrt{2\pi \sigma_2^2}}\exp\left(-\frac{(x_i-\mu_2)^2}{2\sigma_2^2}\right)\left(\frac{x_i-\mu_2}{\sigma_2^2}\right)\\
           \sum_{i=1}^n \frac{1}{f_X(x_i)}\frac{p}{\sqrt{2\pi\sigma_1^2}}\exp\left(-\frac{(x_i-\mu_1)^2}{2\sigma_1^2}\right)\left(\frac{(x_i-\mu_1)^2-\sigma_1^2}{|\sigma_1^3|}\right)\\
           \sum_{i=1}^n \frac{1}{f_X(x_i)}\frac{1-p}{\sqrt{2\pi\sigma_2^2}}\exp\left(-\frac{(x_i-\mu_2)^2}{2\sigma_2^2}\right)\left(\frac{(x_i-\mu_2)^2-\sigma_2^2}{|\sigma_2^3|}\right)
         \end{bmatrix}
  \end{align}$$
  
First we define all the functions as follows:

In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv('/Users/saanvimudholkar/Desktop/data.csv', header=None)
M = data.values.tolist()
X = []
for i in range(0,200):
    X.append(M[i][0])
def normal(x,mu,sigma):
    return (1/np.sqrt(2*np.pi*sigma**2))*np.exp((-(x-mu)**2)/(2*sigma**2))

def f(x,p,mu1,mu2,sigma1,sigma2):
    return p*normal(x,mu1,sigma1)+(1-p)*normal(x,mu2,sigma2)

def likelihood(x,p,mu1,mu2,sigma1,sigma2):
    total=1
    for i in range(0,len(X)):
        total = total*f(X[i],p,mu1,mu2,sigma1,sigma2)
        print(total)
    return total

def loglikelihood(x,p,mu1,mu2,sigma1,sigma2):
    total=0
    for i in range(0,len(X)):
        total=total+np.log(f(X[i],p,mu1,mu2,sigma1,sigma2))
    return total
    
def partial_p(X,p,mu1,mu2,sigma1,sigma2):
    total = 0
    for i in range(0,len(X)):
        total = total+((1/f(X[i],p,mu1,mu2,sigma1,sigma2))*(normal(X[i],mu1,sigma1)-normal(X[i],mu2,sigma2)))
    return total

def partial_mu1(X,p,mu1,mu2,sigma1,sigma2):
    total = 0
    for i in range(0,len(X)):
        total = total+((1/f(X[i],p,mu1,mu2,sigma1,sigma2))*p*normal(X[i],mu1,sigma1)*((X[i]-mu1)/sigma1**2))
    return total

def partial_mu2(X,p,mu1,mu2,sigma1,sigma2):
    total = 0
    for i in range(0,len(X)):
        total = total+((1/f(X[i],p,mu1,mu2,sigma1,sigma2))*(1-p)*normal(X[i],mu2,sigma2)*((X[i]-mu2)/sigma2**2))
    return total

def partial_sigma1(X,p,mu1,mu2,sigma1,sigma2):
    total = 0
    for i in range(0,len(X)):
        total = total+((1/f(X[i],p,mu1,mu2,sigma1,sigma2))*p*normal(X[i],mu1,sigma1)*((((X[i]-mu1)**2)-(sigma1**2))/np.abs(sigma1**3)))
    return total

def partial_sigma2(X,p,mu1,mu2,sigma1,sigma2):
    total = 0
    for i in range(0,len(X)):
        total = total+((1/f(X[i],p,mu1,mu2,sigma1,sigma2))*(1-p)*normal(X[i],mu2,sigma2)*((((X[i]-mu2)**2)-(sigma2**2))/np.abs(sigma2**3)))
    return total

def gradient(X,p,mu1,mu2,sigma1,sigma2):
    return np.array([[partial_p(X,p,mu1,mu2,sigma1,sigma2),partial_mu1(X,p,mu1,mu2,sigma1,sigma2),partial_mu2(X,p,mu1,mu2,sigma1,sigma2),partial_sigma1(X,p,mu1,mu2,sigma1,sigma2),partial_sigma2(X,p,mu1,mu2,sigma1,sigma2)]])

#### Question 1a

Using all the initial estimates, we find the maximum likelihood estimates of $ \lambda = (p,\mu_1,\mu_2,\sigma_1,\sigma_2) $ using ordinary gradient descent method as follows:

In [21]:
p = 0.88
mu1 = 3.79
mu2 = 32.64
sigma1 = 8.08
sigma2 = 7.39

for j in range(0,10000):
    print('Iteration ',j,'; y =(',p,',', mu1,',',mu2,',',sigma1,',',sigma2,')')
    a = p
    b = mu1
    c = mu2
    d = sigma1
    e = sigma2
    p = a + 0.002*partial_p(X,a,b,c,d,e)
    mu1 = b + 0.002*partial_mu1(X,a,b,c,d,e)
    mu2 = c + 0.002*partial_mu2(X,a,b,c,d,e)
    sigma1 = d + 0.002*partial_sigma1(X,a,b,c,d,e)
    sigma2 = e + 0.002*partial_sigma2(X,a,b,c,d,e)
    if np.linalg.norm(gradient(X,p,mu1,mu2,sigma1,sigma2))<=10**-5 or p<0 or p>1:
        print('The maximum likelihood estimates of y = (',p,',', mu1,',',mu2,',',sigma1,',',sigma2,')')
        break
loglikelihood(X,a,b,c,d,e)

Iteration  0 ; y =( 0.88 , 3.79 , 32.64 , 8.08 , 7.39 )
Iteration  1 ; y =( 0.8935611823862428 , 3.790752894586099 , 32.63987413885199 , 8.082315202914131 , 7.3910077321416505 )
Iteration  2 ; y =( 0.8597806350066038 , 3.791715319948181 , 32.63990416538742 , 8.085014123874863 , 7.391846474256193 )
Iteration  3 ; y =( 0.9301049175578072 , 3.792180370916598 , 32.63956062688726 , 8.086759079447265 , 7.393101000822578 )
Iteration  4 ; y =( 0.6944832211632178 , 3.793854718623239 , 32.64005472207549 , 8.090875930611434 , 7.393498674787138 )
The maximum likelihood estimates of y = ( 1.014524035854293 , 3.792675626215902 , 32.63813413565857 , 8.089909034444203 , 7.39692229927185 )


-785.700137266433

#### Question 1b

Using all the initial estimates, we find the maximum likelihood estimates of $ \lambda = (p,\mu_1,\mu_2,\sigma_1,\sigma_2) $ using gradient projection method by projecting estimate of p to the interval $[0,1]$ as follows:

In [22]:
p = 0.88
mu1 = 3.79
mu2 = 32.64
sigma1 = 8.08
sigma2 = 7.39

for j in range(1,10001):
    a = p
    b = mu1
    c = mu2
    d = sigma1
    e = sigma2
    p = p + 0.002*partial_p(X,a,b,c,d,e)
    mu1 = mu1 + 0.002*partial_mu1(X,a,b,c,d,e)
    mu2 = mu2 + 0.002*partial_mu2(X,a,b,c,d,e)
    sigma1 = sigma1 + 0.002*partial_sigma1(X,a,b,c,d,e)
    sigma2 = sigma2 + 0.002*partial_sigma2(X,a,b,c,d,e)
    if j%500==0:
        print('Iteration ',j,'; y =(',p,',', mu1,',',mu2,',',sigma1,',',sigma2,')') 
    if p<0:
        p=0
    elif p>1:
        p =1
    if np.linalg.norm(a-p)<=10**(-5):
        print('The maximum likelihood estimates of y at',j,'is ='' (',p,',', mu1,',',mu2,',',sigma1,',',sigma2,')')
        break

loglikelihood(X,p,mu1,mu2,sigma1,sigma2)

Iteration  500 ; y =( 0.8641186356761509 , 4.7712160256014 , 27.14015430887643 , 9.526534175065848 , 16.729693040284662 )
Iteration  1000 ; y =( 0.8364926959900572 , 3.9486599553341644 , 26.977724852672537 , 8.378893223916217 , 16.216491415244352 )
The maximum likelihood estimates of y at 1330 is = ( 0.8310626273354524 , 3.7035837742088713 , 26.835515296952526 , 8.216631630758425 , 15.847350746279592 )


-769.5181990801111

The estimate of p seems to be slowly decreasing as the program runs. We can observe that the values of $ \mu_1, \mu_2 $ are smaller than the initial values whereas the values of $\sigma_1, \sigma_2$ are larger than the initial estimates. This indicates that the graph is spread wider and the height of the peaks is shorter. Further, the decrease in $\mu_2$ and increase in $\sigma_2$ is much higher than $\mu_1, \sigma_1$ thus, the $\mu_2, \sigma_2 $ peak will be wider and shorter as compared to $\mu_1,\sigma_1$.

#### Question 1c

Using all the initial estimates, we find the maximum likelihood estimates of $ \lambda = (p,\mu_1,\mu_2,\sigma_1,\sigma_2) $ using gradient projection method by projecting estimate of p to the interval $[0.1,0.9]$ as follows:

In [5]:
p = 0.88
mu1 = 3.79
mu2 = 32.64
sigma1 = 8.08
sigma2 = 7.39

for j in range(1,10001):
    a = p
    b = mu1
    c = mu2
    d = sigma1
    e = sigma2
    p = a + 0.002*partial_p(X,a,b,c,d,e)
    mu1 = b + 0.002*partial_mu1(X,a,b,c,d,e)
    mu2 = c + 0.002*partial_mu2(X,a,b,c,d,e)
    sigma1 = d + 0.002*partial_sigma1(X,a,b,c,d,e)
    sigma2 = e + 0.002*partial_sigma2(X,a,b,c,d,e)
    if j%500==0:
        print('Iteration ',j,'; y =(',p,',', mu1,',',mu2,',',sigma1,',',sigma2,')') 
    if p<0.1:
        p=0.1
    elif p>0.9:
        p =0.9
    if np.linalg.norm(a-p)<=10**-5:
        print('The maximum likelihood estimates of y at',j,'is ='' (', mu1,',',mu2,',',sigma1,',',sigma2,')')
        break
print('The algorithm does not converge after 10000 iterations')   

loglikelihood(X,p,mu1,mu2,sigma1,sigma2)

Iteration  500 ; y =( 0.8458549066794072 , 3.9391314254363317 , 32.569247407130156 , 8.32956048880279 , 7.777891032646901 )
Iteration  1000 ; y =( 0.8423154465738639 , 3.9402027670971256 , 32.483679843888275 , 8.324967944149352 , 8.007038490660603 )
Iteration  1500 ; y =( 0.8386500707201983 , 3.922336205807169 , 32.378980263538466 , 8.312540694492844 , 8.169449506793214 )
Iteration  2000 ; y =( 0.8352370507138831 , 3.903755717221643 , 32.2649169897863 , 8.300307074819333 , 8.29592935727593 )
Iteration  2500 ; y =( 0.8320334159241537 , 3.8861652147413133 , 32.14720539046695 , 8.288761148201717 , 8.401301997390046 )
Iteration  3000 ; y =( 0.8289859973824691 , 3.869481532179545 , 32.02881492268164 , 8.277850086765204 , 8.49366267574765 )
Iteration  3500 ; y =( 0.8260574522592197 , 3.8535163607034364 , 31.911209277983293 , 8.267492580325346 , 8.577720739633449 )
Iteration  4000 ; y =( 0.8232217541853063 , 3.8381250527243513 , 31.79503516105767 , 8.257620517974619 , 8.65633075774875 )
Itera

-770.5622003108757

The algorithm does not converge after 10000 iterations which indicates that given the projection condition, the stepsize is too big for the value to converge.

#### Question 1d

Using all the initial estimates, we find the maximum likelihood estimates of $ \lambda = (p,\mu_1,\mu_2,\sigma_1,\sigma_2) $ using gradient projection method by projecting estimate of p to the interval $[0.1,0.9]$ with stepsize = $0.001$ as follows:

In [23]:
p = 0.88
mu1 = 3.79
mu2 = 32.64
sigma1 = 8.08
sigma2 = 7.39

for j in range(0,10000):
    a = p
    b = mu1
    c = mu2
    d = sigma1
    e = sigma2
    p = a + 0.001*partial_p(X,a,b,c,d,e)
    mu1 = b + 0.001*partial_mu1(X,a,b,c,d,e)
    mu2 = c + 0.001*partial_mu2(X,a,b,c,d,e)
    sigma1 = d + 0.001*partial_sigma1(X,a,b,c,d,e)
    sigma2 = e + 0.001*partial_sigma2(X,a,b,c,d,e)
    if p<0.1:
        p=0.1
    elif p>0.9:
        p =0.9
    if np.linalg.norm(a-p)<=10**-5:
        print('The maximum likelihood estimates of y at',j,'is ='' (', mu1,',',mu2,',',sigma1,',',sigma2,')')
        break
        
print('The value of the loglikelihood function is:', loglikelihood(X,p,mu1,mu2,sigma1,sigma2))

The maximum likelihood estimates of y at 17 is = ( 3.7972374556662207 , 32.63931917923823 , 8.101040749857832 , 7.398537388967441 )
The value of the loglikelihood function is: -767.742483870644


On decreasing the stepsize, the code converges giving and MLE which is close to the value of $\lambda$ obtained in $q1a$ which minor differences. Further on calculating the loglikehood of all cases, we see that the values are $value(q1d)>value(q1a)>value(q1b)>value(q1c) $ meaning, given the data set, the conditions and $q1d $ provides the most accurate answer.

## Question 2

#### Question 2a

Let Type A = $ x $, Type B = $ y $ and Type C = $z$, thus from the given question we can formulate the problem as:

$$ min\ -150x-175y-218z\\ such\ that\ 6.5x+7.8y+9.2z<=120\\ x+y+z<=12\\ -45x-60y-75z<=-800\\ -x<=-2\\ -y<=-2 $$

We can also frame the question in a linear problem as:

$$ min\ c^Tx\\ such\ that\ Ax<=b $$

where

$$\begin{align}
   A
      =   
    \begin{bmatrix}
           6.5&7.8&9.2\\
           1&1&1\\
           -45&-60&-75\\
           -1&0&0\\
           0&-1&0\\
         \end{bmatrix}
  \end{align}$$
  
$$\begin{align}
   b
      =   
    \begin{bmatrix}
           120\\
           12\\
           -800\\
           -2\\
           -2\\
         \end{bmatrix}
  \end{align}$$  
  
$$\begin{align}
   c
      =   
    \begin{bmatrix}
           -150\\
           -175\\
           -218\\
         \end{bmatrix}
  \end{align}$$    

From the given question we can formulate the linear programming problem as:

Let $p=(x,y,z)^t$ and $\lambda=(\lambda_1,\lambda_2,\lambda_3, \lambda_4, \lambda_5)$

$ L(p,\lambda)=-150x-175y-218z+\lambda_1(6.5x+7.8y+9.2z-120)+\lambda_2(x+y+z-12)-\lambda_3(45x+60y+75z-800)-\lambda_4(x-2)-\lambda_5(y-2) $

OR

$$ L(p,\lambda) = (c^T+\lambda^T A)x - \lambda^T b $$

#### Question 2b

We can get the KKT conditions as follows:

$$ dL(p,\lambda)/dx=>-150+6.5\lambda_1+\lambda_2-45\lambda_3-\lambda_4=0\\ dL(p,\lambda)/dy=>-175+7.8\lambda_1+\lambda_2-60\lambda_3-\lambda_5=0\\ dL(p,\lambda)/dz=>-218+9.2\lambda_1+\lambda_2-75\lambda_3=0$$

The complimentary slackness conditions are as follows:

$$ \lambda_1(6.5x+7.8y+9.2z-120)=0\\ \lambda_2(x+y+z-12)=0\\ \lambda_3(-45x-60y-75z+800)=0\\ \lambda_4(-x+2)=0\\ \lambda_5(-y+2)=0 $$

From the conditions, we consider the case $$ \lambda_2=\lambda_4=\lambda_5 \neq 0 $$ thus by complimentary slackness conditions we can say that:
$$ x+y+z-12=0\\ -x+2=0\\ -y+2=0 $$
Thus, $$x=2\\ y=2\\ z=8 $$
Consider, $\lambda_1=\lambda_3 =0 $, thus $6.5x+7.8y+9.2z-120 \neq 0$ and $-45x-60y-75z+800 \neq 0$ respectively
Further on substituting $x,y,z$ in the above equations, 
$$6.5x+7.8y+9.2z-120 => 102.2-120=> -17.8 \neq 0 $$
$$-45x-60y-75z+800 => -810+800 => -10 \neq 0 $$
Thus proving the above statement 



For optimality conditions, on substituting $\lambda_1=\lambda_3 =0 $ we get,
$$ -150+\lambda_2-\lambda_4=0\\ -175+\lambda_2-\lambda_5=0\\ -218+\lambda_2=0 $$

Thus, $\lambda_2=218, \lambda_4=68, \lambda_5=43$ which also satisfies the condition, $\lambda_2=\lambda_4=\lambda_5 \neq 0 $

Lastly, as $ \lambda_1,\lambda_2,\lambda_3,\lambda_4,\lambda_5>=0$, and satisfying the both optimality conditions and complimnetary slackness conditions, we can apply the Theorem: KKT Conditions for Convex Linearly Constrained Problems - Necessary and Sufficient Optimality Conditions, to say that 
$$p=(x,y,z)=(2,2,8)$$
is an optimal solution.

#### Question 2c

According to the dual objective function

$$ q(\lambda) = min L(x,\lambda)\\= min (c^T+\lambda^T A)x - \lambda^T b\\= -\lambda^T b $$ if $$c+A^T \lambda>=0\\ -\infty  otherwise $$

Thus we can formulate the dual problem as:

$$ max -\lambda^T b\\= -120\lambda_1-12\lambda_2+800\lambda_3+2\lambda_4+2\lambda_5 $$

such that,

$$6.5\lambda_1+\lambda_2-45\lambda_3-\lambda_4 = 150\\ 7.8\lambda_1+\lambda_2-60\lambda_3-\lambda_5 = 175\\ 9.2\lambda_1+\lambda_2-75\lambda_3 = 218 $$

#### Question 2d

In [17]:
y = np.array([[5],[5],[5],[5],[5]])
t = 0.005
A = np.array([[6.5, 1, -45,-1,0],[7.8,1,-60,0,-1],[9.2,1,-75,0,0]])
c = np.array([[-120],[-12],[800],[2],[2]])
b = np.array([[150],[175],[218]])
grad = c
for i in range(1,400001):
    m = y + t*grad
    y = m - (A.transpose()@(np.linalg.inv(A@A.transpose()))@((A@m)-b))
    for j in range(0,5):
        if y[j]<=0:
            y[j]=0
    if i%20000==0:
        print('Iteration ',i,'; y = ', y)
print('Value of lambda:',y)

Iteration  20000 ; y =  [[  0.        ]
 [176.82947536]
 [  0.        ]
 [ 51.50237315]
 [ 34.75268744]]
Iteration  40000 ; y =  [[  0.        ]
 [213.01157484]
 [  0.        ]
 [ 65.92051966]
 [ 41.96455117]]
Iteration  60000 ; y =  [[  0.        ]
 [214.958124  ]
 [  0.        ]
 [ 66.6961969 ]
 [ 42.35253991]]
Iteration  80000 ; y =  [[  0.        ]
 [215.06284578]
 [  0.        ]
 [ 66.73792731]
 [ 42.3734132 ]]
Iteration  100000 ; y =  [[  0.        ]
 [215.06847967]
 [  0.        ]
 [ 66.74017235]
 [ 42.37453615]]
Iteration  120000 ; y =  [[  0.        ]
 [215.06878277]
 [  0.        ]
 [ 66.74029313]
 [ 42.37459657]]
Iteration  140000 ; y =  [[  0.        ]
 [215.06879907]
 [  0.        ]
 [ 66.74029963]
 [ 42.37459982]]
Iteration  160000 ; y =  [[  0.        ]
 [215.06879995]
 [  0.        ]
 [ 66.74029998]
 [ 42.37459999]]
Iteration  180000 ; y =  [[  0.    ]
 [215.0688]
 [  0.    ]
 [ 66.7403]
 [ 42.3746]]
Iteration  200000 ; y =  [[  0.    ]
 [215.0688]
 [  0.    ]
 [ 66.740

The values of $\lambda$ obtained in part d is slightly different from part b. For $\lambda_2$ the difference is of 3 digits, for $\lambda_4$ difference is of 2 digits and for $\lambda_5$ difference is of 1 digit. For $\lambda_1,\lambda_3$ the values are the same which is 0.

Thus, we do not get the same answer as 2b but close.

Further, upon decreasing the stepsize, we get values closer to the answer in 2b. Thus, to improve the precision, we must decrease the stepsize